In [1]:
from db_utils import retrieve_data
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
path = r"c:\Users\brtnl\OneDrive\Desktop\code\sports_betting"
os.chdir(path)

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [3]:
useless_cols = ["ot", "unique_id", "created_at"]

In [4]:
def add_temporal_features(df):
    df['date'] = pd.to_datetime(df['date'])
    df['days_since'] = (pd.to_datetime("today") - df['date']).dt.days
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df.drop(['date'], axis=1, inplace=True)
    return df

In [5]:
# basic ideas, maybe some month? month * indicator? 
def team_name(df):
    one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')
    one_hot_encoded = one_hot_encoder.fit_transform(df[["home", "visitor"]])
    feature_names = one_hot_encoder.get_feature_names_out(["home", "visitor"])
    df_one_hot = pd.DataFrame(one_hot_encoded, columns=feature_names)
    df_encoded = pd.concat([df.drop(["home", "visitor"], axis=1), df_one_hot], axis=1)
    return df_encoded

In [6]:
query = """SELECT * FROM cfb.all_games"""
game_df = retrieve_data(query)

In [7]:
game_df["home_half"] = game_df["home_first_quarter"] + game_df["home_second_quarter"]
game_df["visitor_half"] = game_df["visitor_first_quarter"] + game_df["visitor_second_quarter"]

In [8]:
# game_df = team_name(game_df)
game_df = add_temporal_features(game_df)
game_df = team_name(game_df)
game_df.drop(columns=useless_cols, axis=1, inplace=True)

In [14]:
X.fillna(0, inplace=True)

C:\Users\brtnl\AppData\Local\Temp\ipykernel_11664\2534066105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [9]:
target_col = "home_second_quarter"
X = game_df.loc[:, game_df.columns != target_col]
y = game_df[target_col]

In [16]:
first_model = LinearRegression().fit(X, y)

In [ ]:
# Indicators for the home and visitor team  
game_df = pd.concat([game_df, pd.get_dummies(game_df["home"], prefix="home"), pd.get_dummies(game_df["visitor"], prefix="vis")], axis=1)

In [ ]:
categorical_cols = ["date", "home", "visitor", "ot", "unique_id", "created_at"]
game_df.loc[:, ~game_df.columns.isin(categorical_cols)].fillna(0)

,home_points,visitor_points,home_first_quarter,home_second_quarter,home_third_quarter,home_fourth_quarter,home_ot_total,visitor_first_quarter,visitor_second_quarter,visitor_third_quarter,visitor_fourth_quarter,visitor_ot_total,home_rank,visitor_rank,home_half,visitor_half,home_Air Force,home_Akron,home_Alabama,home_Appalachian State,home_Arizona,home_Arizona State,home_Arkansas,home_Arkansas State,home_Army,home_Auburn,home_Ball State,home_Baylor,home_Boise State,home_Boston College,home_Bowling Green State,home_Brigham Young,home_Buffalo,home_California,home_Campbell,home_Central Michigan,home_Charleston Southern,home_Charlotte,home_Cincinnati,home_Clemson,...,vis_Texas Tech,vis_The Citadel,vis_Toledo,vis_Towson,vis_Troy,vis_Tulane,vis_Tulsa,vis_UAB,vis_UCF,vis_UCLA,vis_UNLV,vis_USC,vis_UTEP,vis_UTSA,vis_Utah,vis_Utah State,vis_Utah Tech,vis_Valparaiso,vis_Vanderbilt,vis_Villanova,vis_Virginia,vis_Virginia Military Institute,vis_Virginia Tech,vis_Wagner,vis_Wake Forest,vis_Washington,vis_Washington State,vis_Weber State,vis_Wesley,vis_West Virginia,vis_Western Carolina,vis_Western Illinois,vis_Western Kentucky,vis_Western Michigan,vis_William & Mary,vis_Wisconsin,vis_Wofford,vis_Wyoming,vis_Yale,vis_Youngstown State
0,54,52,7,17,0,22,8,10,7,21,8,6,0.0,0.0,24,17,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,30,26,14,0,10,6,0,3,14,6,3,0,0.0,0.0,14,17,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,17,34,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,53,7,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,40,35,0,0,0,0,0,0,0,0,0,0,0.0,8.0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,28,29,0,0,0,0,0,0,0,0,0,0,4.0,0.0,0,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F